## Práctica 2: Transformaciones geométricas
Integrantes: David García Díaz y Alejandro Bolaños García

## 1a. Desarrollar una aplicación que lleve a cabo transformaciones de la imagen en tiempo real a través de una interfaz basada en trackbars o equivalente.


In [21]:
import cv2 as cv
import numpy as np

# Variables globales
tx, ty = 0, 0
angle = 0
scale_x = 1.0
scale_y = 1.0
size = (1000, 1000)
image = cv.imread('images/cats.png', cv.IMREAD_COLOR)

# Verificar si la imagen se cargó correctamente
if image is None:
    raise FileNotFoundError("La imagen no se pudo cargar. Verifica la ruta del archivo.")

# Tamaño de la imagen original
h, w = image.shape[:2]

def update_image():
    global tx, ty, angle, scale_x, scale_y

    # Centro de la imagen para la rotación
    center = (w / 2, h / 2)

    # Crear la matriz de transformación afín
    M = cv.getRotationMatrix2D(center, angle, 1.0)
    
    # Ajustar la matriz para incluir el escalado
    M[0, 0] *= scale_x
    M[0, 1] *= scale_x
    M[1, 0] *= scale_y
    M[1, 1] *= scale_y

    # Añadir traslación
    M[0, 2] += tx
    M[1, 2] += ty

    # Aplicar la transformación afín
    transformed_image = cv.warpAffine(image, M, size)
    cv.imshow('Transformaciones', transformed_image)

def on_trackbar_rotation(val):
    global angle
    angle = val
    update_image()

def on_trackbar_translation(val):
    global tx, ty
    tx = val - 100  # Permitir traslación negativa
    ty = val - 100  # Permitir traslación negativa
    update_image()

def on_trackbar_resize(val):
    global scale_x, scale_y
    scale_x = val / 100
    scale_y = val / 100
    update_image()

def on_trackbar_resize_x(val):
    global scale_x
    scale_x = val / 100
    update_image()

def on_trackbar_resize_y(val):
    global scale_y
    scale_y = val / 100
    update_image()

# Crear ventana y barras deslizantes
cv.namedWindow('Transformaciones')
cv.createTrackbar('Rotation', 'Transformaciones', 0, 360, on_trackbar_rotation)
cv.createTrackbar('Translation', 'Transformaciones', 100, 200, on_trackbar_translation)  # Centrar en 0
cv.createTrackbar('Resize', 'Transformaciones', 100, 200, on_trackbar_resize)
cv.createTrackbar('Resize X', 'Transformaciones', 100, 200, on_trackbar_resize_x)
cv.createTrackbar('Resize Y', 'Transformaciones', 100, 200, on_trackbar_resize_y)

# Inicializar la imagen con valores predeterminados
update_image()

cv.waitKey(0)
cv.destroyAllWindows()


## 1b. Dada una imagen trazar una ventana de proyección y proyectar la imagen.

In [106]:
import cv2 as cv
import numpy as np

# Cargar la imagen
image = cv.imread('images/cats.png', cv.IMREAD_COLOR)

# Verificar si la imagen se cargó correctamente
if image is None:
    raise FileNotFoundError("La imagen no se pudo cargar. Verifica la ruta del archivo.")

# Tamaño de la imagen original
h, w = image.shape[:2]

# Definir los puntos de la ventana de proyección en la imagen original (cuatro esquinas)
src_points = np.float32([[0, 0], [w, 0], [w, h], [0, h]])

# Lista para almacenar los puntos de destino
dst_points = []

# Copia de la imagen para dibujar los puntos seleccionados
image_copy = image.copy()

# Función para manejar los eventos del mouse
def mouse(event, x, y, flags, param):
    global dst_points, image

    if event == cv.EVENT_LBUTTONDOWN:
        if len(dst_points) < 4:
            # Almacenar el punto seleccionado
            dst_points.append([x, y])
            # Dibujar un círculo en la posición seleccionada
            cv.circle(image_copy, (x, y), 10, (0, 255, 0), -1)
            cv.imshow('Imagen original', image_copy)

        # Si se seleccionaron 4 puntos, aplicar la transformación
        if len(dst_points) == 4:
            dst_points_np = np.float32(dst_points)

            # Calcular la matriz de transformación de perspectiva
            M = cv.getPerspectiveTransform(src_points, dst_points_np)

            # Aplicar la transformación de perspectiva
            projected_image = cv.warpPerspective(image, M, (w, h))

            # Mostrar la imagen proyectada
            cv.imshow('Imagen Proyectada', projected_image)

# Mostrar la imagen y configurar la función de callback del mouse
cv.imshow('Imagen original', image)
cv.setMouseCallback('Imagen original', mouse)

cv.waitKey(0)
cv.destroyAllWindows()


## 1c. Desarrollar una aplicación que lleve a cabo distorsiones de la lente. Para ello los coeficientes de distorsión deben gobernarse a través de una interfaz

In [7]:
import cv2 as cv
import numpy as np

# Cargar la imagen
image = cv.imread('images/cats.png', cv.IMREAD_COLOR)

# Verificar si la imagen se cargó correctamente
if image is None:
    raise FileNotFoundError("La imagen no se pudo cargar. Verifica la ruta del archivo.")

# Tamaño de la imagen
h, w = image.shape[:2]

# Definir los coeficientes iniciales de distorsión
dist_coeffs = np.zeros((4,1), np.float64)

# Matriz de la cámara (suponiendo una matriz de identidad)
camera_matrix = np.array([[w, 0, w / 2],
                          [0, h, h / 2],
                          [0, 0, 1]], dtype=np.float32)

# Función para actualizar la imagen distorsionada
def update_image():
    global dist_coeffs, camera_matrix, image
    
    # Aplicar la distorsión usando los coeficientes actuales
    distorted_image = cv.undistort(image, camera_matrix, dist_coeffs)
    
    # Mostrar la imagen distorsionada
    cv.imshow('Distorsion de la Lente', distorted_image)

# Funciones para los eventos de los trackbars
def on_trackbar_k1(val):
    # Ajustar el valor para que sea entre -1.0 y 1.0
    dist_coeffs[0, 0] = (val - 1000) / 1000.0
    update_image()

def on_trackbar_k2(val):
    # Ajustar el valor para que sea entre -1.0 y 1.0
    dist_coeffs[1, 0] = (val - 1000) / 1000.0
    update_image()

# Crear una ventana para mostrar la imagen
cv.namedWindow('Distorsion de la Lente')

# Crear los trackbars para ajustar los coeficientes de distorsión
cv.createTrackbar('K1', 'Distorsion de la Lente', 1000, 2000, on_trackbar_k1)
cv.createTrackbar('K2', 'Distorsion de la Lente', 1000, 2000, on_trackbar_k2)

update_image()

cv.waitKey(0)
cv.destroyAllWindows()


#### Opcional: 
- Marcar punto de giro con ratón
- Arrastrar imagen con ratón
- Transformaciones iniclaes 
- Todo para imagen y vídeo

In [2]:
import cv2 as cv
import numpy as np

# Variables globales
tx, ty = 0, 0
angle = 0
scale_x = 1.0
scale_y = 1.0
center = None
dragging = False
prev_x, prev_y = 0, 0

# Define si quieres usar una imagen o un vídeo
USE_IMAGE = True  # Cambia a False para usar vídeo en tiempo real
IMAGE_PATH = 'images/cats.png'

# Carga la imagen o el vídeo en función de la variable USE_IMAGE
if USE_IMAGE:
    image = cv.imread(IMAGE_PATH, cv.IMREAD_COLOR)
    if image is None:
        raise FileNotFoundError("La imagen no se pudo cargar. Verifica la ruta del archivo.")
    size = (image.shape[1], image.shape[0])
else:
    cap = cv.VideoCapture(0)
    if not cap.isOpened():
        raise IOError("No se puede acceder a la cámara.")
    size = (640, 480)

def update_image(frame):
    global tx, ty, angle, scale_x, scale_y, center

    h, w = frame.shape[:2]

    # Centro por defecto (si no se ha seleccionado uno con el ratón)
    if center is None:
        center = (w // 2, h // 2)

    # Crear la matriz de transformación afín
    M = cv.getRotationMatrix2D(center, angle, 1.0)

    # Ajustar la matriz para incluir el escalado
    M[0, 0] *= scale_x
    M[0, 1] *= scale_x
    M[1, 0] *= scale_y
    M[1, 1] *= scale_y

    # Añadir traslación
    M[0, 2] += tx
    M[1, 2] += ty

    # Aplicar la transformación afín
    transformed_image = cv.warpAffine(frame, M, size)
    cv.imshow('Transformaciones', transformed_image)

def on_trackbar_rotation(val):
    global angle
    angle = val
    update_image(image if USE_IMAGE else frame)

def on_trackbar_resize(val):
    global scale_x, scale_y
    scale_x = val / 100
    scale_y = val / 100
    update_image(image if USE_IMAGE else frame)

def on_trackbar_resize_x(val):
    global scale_x
    scale_x = val / 100
    update_image(image if USE_IMAGE else frame)

def on_trackbar_resize_y(val):
    global scale_y
    scale_y = val / 100
    update_image(image if USE_IMAGE else frame)

# Funciones de manejo del ratón
def on_mouse(event, x, y, flags, param):
    global center, tx, ty, prev_x, prev_y, dragging
    
    if event == cv.EVENT_LBUTTONDOWN:
        # Definir el nuevo centro de rotación
        center = (x, y)
    
    elif event == cv.EVENT_MOUSEMOVE:
        if dragging:
            dx = x - prev_x
            dy = y - prev_y
            tx += dx
            ty += dy
            prev_x, prev_y = x, y
            update_image(image if USE_IMAGE else frame)
    
    elif event == cv.EVENT_LBUTTONUP:
        dragging = False
    
    elif event == cv.EVENT_LBUTTONDBLCLK:
        dragging = True
        prev_x, prev_y = x, y

# Crear ventana y barras deslizantes
cv.namedWindow('Transformaciones')
cv.createTrackbar('Rotation', 'Transformaciones', 0, 360, on_trackbar_rotation)
cv.createTrackbar('Resize', 'Transformaciones', 100, 200, on_trackbar_resize)
cv.createTrackbar('Resize X', 'Transformaciones', 100, 200, on_trackbar_resize_x)
cv.createTrackbar('Resize Y', 'Transformaciones', 100, 200, on_trackbar_resize_y)
cv.setMouseCallback('Transformaciones', on_mouse)

# Bucle de ejecución
if USE_IMAGE:
    # Para imagen estática, solo necesitamos actualizar la imagen una vez
    update_image(image)
    cv.waitKey(0)
else:
    # Para vídeo, repetimos en cada fotograma
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        update_image(frame)
        if cv.waitKey(1) & 0xFF == 27:  # Presionar 'ESC' para salir
            break
    cap.release()

cv.destroyAllWindows()


#### Dada una imagen seleccionar tres puntos de la imagen original y tres puntos en una imagen destino y realizar la transformación afín.

In [129]:
import cv2 as cv
import numpy as np

# Cargar las imágenes
image_src = cv.imread('images/cats.png', cv.IMREAD_COLOR)  # Imagen de origen
image_dst = cv.imread('images/cats.png', cv.IMREAD_COLOR)  # Imagen de destino

# Verificar si las imágenes se cargaron correctamente
if image_src is None or image_dst is None:
    raise FileNotFoundError("Una o ambas imágenes no se pudieron cargar. Verifica las rutas.")

# Listas para almacenar los puntos seleccionados
src_points = []
dst_points = []

# Copias de las imágenes para dibujar los puntos seleccionados
image_src_copy = image_src.copy()
image_dst_copy = image_dst.copy()

# Función para manejar los eventos del mouse en la imagen de origen
def mouse_src(event, x, y, flags, param):
    global src_points

    if event == cv.EVENT_LBUTTONDOWN and len(src_points) < 3:
        # Almacenar el punto seleccionado
        src_points.append([x, y])
        # Dibujar un círculo en la posición seleccionada
        cv.circle(image_src_copy, (x, y), 5, (0, 255, 0), -1)
        cv.imshow('Imagen Origen', image_src_copy)

# Función para manejar los eventos del mouse en la imagen de destino
def mouse_dst(event, x, y, flags, param):
    global dst_points

    if event == cv.EVENT_LBUTTONDOWN and len(dst_points) < 3:
        # Almacenar el punto seleccionado
        dst_points.append([x, y])
        # Dibujar un círculo en la posición seleccionada
        cv.circle(image_dst_copy, (x, y), 5, (0, 0, 255), -1)
        cv.imshow('Imagen Destino', image_dst_copy)

    # Si ya se han seleccionado tres puntos en ambas imágenes, aplicar la transformación afín
    if len(src_points) == 3 and len(dst_points) == 3:
        src_points_np = np.float32(src_points)
        dst_points_np = np.float32(dst_points)

        # Calcular la matriz de transformación afín
        M = cv.getAffineTransform(src_points_np, dst_points_np)

        # Aplicar la transformación afín en la imagen de origen
        h, w = image_dst.shape[:2]
        transformed_image = cv.warpAffine(image_src, M, (w, h))

        # Mostrar la imagen transformada
        cv.imshow('Imagen Transformada', transformed_image)

# Mostrar la imagen de origen y configurar la función de callback del mouse
cv.imshow('Imagen Origen', image_src)
cv.setMouseCallback('Imagen Origen', mouse_src)

# Mostrar la imagen de destino y configurar la función de callback del mouse
cv.imshow('Imagen Destino', image_dst)
cv.setMouseCallback('Imagen Destino', mouse_dst)

cv.waitKey(0)
cv.destroyAllWindows()


#### Calcular la imagen especular a partir de una imagen.

In [11]:
import cv2 as cv

# Cargar la imagen
image = cv.imread('images/cats.png', cv.IMREAD_COLOR)

# Verificar si la imagen se cargó correctamente
if image is None:
    raise FileNotFoundError("La imagen no se pudo cargar. Verifica la ruta del archivo.")

# Calcular la imagen especular (reflejo horizontal)
mirror_image = cv.flip(image, 1)  # El segundo parámetro 1 indica voltear horizontalmente

# Mostrar la imagen original y la imagen especular
cv.imshow('Imagen Original', image)
cv.imshow('Imagen Especular', mirror_image)

# Esperar hasta que el usuario presione una tecla
cv.waitKey(0)
cv.destroyAllWindows()
